This file is the companion of the article:
IMU Error Modeling Tutorial: INS state estimation and real-time sensor calibration 
authored by 
Jay A. Farrell, Felipe O. Silva, Farzana Rahman, and J. Wendel.
This tutorial is accepted for publication in 
IEEE Control Systems Magazine (referred to below as IEEE CSM)
The articles main point-of-contact is J. Farrell (farrell@ece.ucr.edu)

This software is distributed for academic purposes under the MIT License
 Copyright (c) 2025 JAY A FARRELL
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
 
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
 
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


# Purpose
This notebook defines is an example of a first-order Gauss-Markov error model that matches the Allan standard deviation plot for delays up to about 100 seconds. This is more than sufficient for tightly-integrated GNSS aided INS applications where the INS is typically corrected by the GNSS measurements at 1 Hz, but may miss GNSS corrections over short intervals of time. When those intervals grow beyond 10 seconds, the application behavior should be concerned about the missing GNSS correction data and grwoing inaccuracy of the INS solution. 

# Assumed Application
The assumtions are:
1) A stochastic estimator is being constructed, such as a Kalman filter, for an aided INS. The estimator requires a state-space error model in continuous and discrete time for each IMU output. See for example, eqn. (11.106) or (11.107) in [2].
2) Each state-space model must be generate data capable of matching an instruments Allan standard deviation plot over a range of delays.  

# Status
This notebook is not quite debugged. The theoretically computed standard deviation should match the emperical standard deviation. Currently is noy quite matching for $t\in[5,10]$. 

# References
This file is the companion of the article:<br>
[1] Jay A. Farrell, Felipe O. Silva, Farzana Rahman, and J. Wendel. 
    "IMU Error Modeling Tutorial: INS state estimation and real-time sensor calibration
    IEEE Control Systems Magazine.

A supplementary reference is:<br>
[2] J. A. Farrell, "Aided Navigation: GPS with High Rate Sensors", McGraw-Hill, 2008.

In [ ]:
import ASD_to_GaussMarkovFirstOrder as ASD2GM1
import allantools
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

# The values of Fs, N, B, Tp are read from the ASD plot and adjusted as needed.
Fs = 100    # Sampling frequency, Hz
N  = 3.3e-3 # ASD, m/s/s/rtHz = m/s/rtsec
B  = 9e-4   # bias instability, m/s/s 
Tp = 300    # desired delay for the GM ASD peak, seconds

# Create an instance of the ASD_to_GaussMarkovFirstOrder class
gm1 = ASD2GM1.ASD_to_GaussMarkovFirstOrder(N, B, Tp, Fs)

In [ ]:
show_plots = False
duration = 10000  # Duration of the simulation in seconds
err, bias, rw = gm1.simulate_discrete_time_model(duration, show_plots)


In [ ]:
# Load the IMU data from the MATLAB file
path = '../AV_Matlab_SW_IEEECSM/parsed_isolated_marble_data_az.mat'
print(f"Loading data from {path}"  )
data = sio.loadmat(path)
acc_z = data['acc_z']
print(f"acc_z shape: {acc_z.shape}")

# Define the time intervals for Allan deviation
vals = np.arange(1, 10, dtype=float)  # Create an array of values from 1 to 9
# Define the time intervals for Allan deviation
taus = np.concatenate([0.01*vals, 0.1*vals, vals, 10*vals, 100*vals])  # Concatenate to create a range of taus

# Calculate Allan deviation for the IMU data
tau, adev, adeverr, n = allantools.oadev(acc_z, rate=Fs, data_type='freq', taus=taus)

# calculate Allan deviation for the simulated data
tau_err, adev_err, adeverr, n = allantools.oadev(err, rate=Fs, data_type='freq', taus=taus)
tau_bias, adev_bias, adeverr, n = allantools.oadev(bias, rate=Fs, data_type='freq', taus=taus)
tau_rw, adev_rw, adeverr, n = allantools.oadev(rw, rate=Fs, data_type='freq', taus=taus)


In [ ]:
# Plot the Allan deviation
fig = plt.loglog(tau, adev, 'o', label='Data')
fig = plt.loglog(tau_bias, adev_bias, '.', label='bias')
fig = plt.loglog(tau_rw, adev_rw, '.', label='random walk')
fig = plt.loglog(tau_err, adev_err, '.', label='total error')
#print(f"tau = {tau}, \nadev = {adev}, \nadeverr = {adeverr}, \nn = {n}")
plt.xlabel('Delay, seconds')
plt.ylabel('ASD, m/s^2')
plt.minorticks_on()
plt.grid(which='both', axis='both')
plt.xlim(left=1e-2, right=5e2)
plt.ylim(bottom=1e-4, top=1e-1)
plt.legend()
plt.show()

# Purpose
For the simulated data and then for the actual data, the next cell computes and plots:
1) instances of error trajectory (narrow transparent curves);
2) the robust standard deviation of those instances at each time instant (wide, opaque, blue curve); and,
3) the theroetically computed standard deviation (wide, opaque, black curve).

In [ ]:
T_avg = 10  # Average time, seconds
T_int = 10  # Integration time, seconds 
# For no overlab:
T_shft = (T_avg + T_int)  # Shift time, seconds
MaxRep = 1000  # Maximum number of repetitions for the covariance test

# Test the covariance function with the simulated data 
gm1.test_cov(err, T_avg, T_int, T_shft, MaxRep)

# Test the covariance function with the real data
gm1.test_cov(acc_z, T_avg, T_int, T_shft, MaxRep)
